In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pandas as pd
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

# Datasets Used to Demonstrate Join Types in Spark

In [2]:
stores = sc.parallelize([(100, 'Boca Raton'),
 (101, 'Columbia'),
 (102, 'Cambridge'),
 (103, 'Naperville')])
# stores schema (store_id, store_location)
salespeople = sc.parallelize([(1, 'Henry', 100),
 (2, 'Karen', 100),
 (3, 'Paul', 101),
 (4, 'Jimmy', 102),
 (5, 'Janice', None)])
# salespeople schema (salesperson_id, salesperson_name, store_id)

# join()
Syntax: RDD.join(<'otherRDD'>, numPartitions=None)

Một triển khai của phép nối bên trong, so khớp hai cặp key/value RDD theo khóa của chúng.

In [4]:
salespeople.keyBy(lambda x: x[2]) \
           .join(stores).collect()

[(100, ((1, 'Henry', 100), 'Boca Raton')),
 (100, ((2, 'Karen', 100), 'Boca Raton')),
 (101, ((3, 'Paul', 101), 'Columbia')),
 (102, ((4, 'Jimmy', 102), 'Cambridge'))]

# leftOuterJoin()
Syntax: RDD.leftOuterJoin(<'otherRDD'>, numPartitions=None)

Trả về tất cả các phần tử hoặc bản ghi từ RDD đầu tiên được tham chiếu. Nếu các khóa từ RDD đầu tiên (hoặc bên trái) có trong RDD bên phải, sau đó bản ghi RDD bên phải được trả về cùng với RDD bên trái ghi lại. Nếu không, bản ghi RDD bên phải là Không (trống).

In [10]:
salespeople.keyBy(lambda x: x[2]) \
           .leftOuterJoin(stores) \
           .collect()

[(None, ((5, 'Janice', None), None)),
 (100, ((1, 'Henry', 100), 'Boca Raton')),
 (100, ((2, 'Karen', 100), 'Boca Raton')),
 (101, ((3, 'Paul', 101), 'Columbia')),
 (102, ((4, 'Jimmy', 102), 'Cambridge'))]

In [11]:
salespeople.keyBy(lambda x: x[2]) \
           .leftOuterJoin(stores) \
           .filter(lambda x: x[1][1] is None) \
           .map(lambda x: "salesperson " + x[1][0][1] + " has no store") \
           .collect()

['salesperson Janice has no store']

# rightOuterJoin()
Syntax: RDD.rightOuterJoin(<'otherRDD'>, numPartitions=None)

Trả về tất cả các phần tử hoặc bản ghi từ RDD thứ hai được tham chiếu. Nếu có các khóa từ RDD thứ hai (hoặc bên phải)trong RDD bên trái, sau đó bản ghi RDD bên trái được trả về cùng với RDD bên phải ghi lại. Nếu không, bản ghi RDD bên trái là Không (trống).

In [12]:
salespeople.keyBy(lambda x: x[2]) \
           .rightOuterJoin(stores) \
           .collect()

[(100, ((1, 'Henry', 100), 'Boca Raton')),
 (100, ((2, 'Karen', 100), 'Boca Raton')),
 (101, ((3, 'Paul', 101), 'Columbia')),
 (102, ((4, 'Jimmy', 102), 'Cambridge')),
 (103, (None, 'Naperville'))]

In [13]:
salespeople.keyBy(lambda x: x[2]) \
           .rightOuterJoin(stores) \
           .filter(lambda x: x[1][0] is None) \
           .map(lambda x: x[1][1] + " store has no salespeople") \
           .collect()

['Naperville store has no salespeople']

# fullOuterJoin()
Syntax: RDD.fullOuterJoin(<'otherRDD'>, numPartitions=None)

Biến đổi tất cả các phần tử từ cả hai RDD cho dù có khóa khớp hay không. Các phím không khớp từ bên trái hoặc bên phảitập dữ liệu được biểu thị là Không có (trống).

In [14]:
salespeople.keyBy(lambda x: x[2]) \
           .fullOuterJoin(stores) \
           .filter(lambda x: x[1][0] is None or x[1][1] is None) \
           .collect()

[(None, ((5, 'Janice', None), None)), (103, (None, 'Naperville'))]

# cogroup()
Syntax:RDD.cogroup(<'otherRDD'>, numPartitions=None)
    
Phép biến đổi cogroup () nhóm nhiều bộ dữ liệu cặp key/value bằng một Key. 

Về mặt khái niệm, nó hơi giống với fullOuterJoin(), nhưng ở đólà một số khác biệt chính trong việc triển khai:

1.Phép biến đổi cogroup() trả về một đối tượng có thể lặp lại, tương tự nhưđối tượng được trả về từ hàm groupByKey().

2.Phép biến đổi cogroup() nhóm nhiều phần tử từ cả hai RDD thành các đối tượng có thể lặp lại, trong khi fullOuterJoin() tạo ra các phần tử đầu ra cho cùng một khóa.

3.Phép biến đổi cogroup() có thể nhóm ba hoặc nhiều RDD bằng cách sử dụng API Scala hoặc bí danh hàm groupWith().

In [15]:
salespeople.keyBy(lambda x: x[2]) \
 .cogroup(stores).take(1)
# returns:
# [(None, (<pyspark.resultiterable.ResultIterable object at ...>,
# <pyspark.resultiterable.ResultIterable object at ...>))]

[(None,
  (<pyspark.resultiterable.ResultIterable at 0x1974aa30ca0>,
   <pyspark.resultiterable.ResultIterable at 0x1974aa30670>))]

In [16]:
salespeople.keyBy(lambda x: x[2]) \
 .cogroup(stores) \
 .mapValues(lambda x: [item for sublist in x for item in sublist]) \
 .collect()
# using the mapValues() to process the Iterable object returns:
# [(None, [(5, 'Janice', None)]),
# (100, [(1, 'Henry', 100), (2, 'Karen', 100), 'Boca Raton']),
# (102, [(4, 'Jimmy', 102), 'Cambridge']), (101, [(3, 'Paul', 101), 'Columbia']),
# (103, ['Naperville'])]

[(None, [(5, 'Janice', None)]),
 (100, [(1, 'Henry', 100), (2, 'Karen', 100), 'Boca Raton']),
 (101, [(3, 'Paul', 101), 'Columbia']),
 (102, [(4, 'Jimmy', 102), 'Cambridge']),
 (103, ['Naperville'])]

# cartesian()
Syntax: RDD.cartesian(<'otherRDD'>)

Phép biến đổi Cartesian() kết hợp chéo, tạo mọi kết hợp bản ghi có thể có từ cả hai RDDs. Số lượng bản ghi được tạo ra bởi sự chuyển đổi này bằng số lượng bản ghi trong RDD đầu tiên nhân với số lượng bản ghi trong RDD thứ hai.

In [17]:
salespeople.keyBy(lambda x: x[2]) \
 .cartesian(stores).collect()

[((100, (1, 'Henry', 100)), (100, 'Boca Raton')),
 ((100, (1, 'Henry', 100)), (101, 'Columbia')),
 ((100, (1, 'Henry', 100)), (102, 'Cambridge')),
 ((100, (1, 'Henry', 100)), (103, 'Naperville')),
 ((100, (2, 'Karen', 100)), (100, 'Boca Raton')),
 ((100, (2, 'Karen', 100)), (101, 'Columbia')),
 ((100, (2, 'Karen', 100)), (102, 'Cambridge')),
 ((100, (2, 'Karen', 100)), (103, 'Naperville')),
 ((101, (3, 'Paul', 101)), (100, 'Boca Raton')),
 ((101, (3, 'Paul', 101)), (101, 'Columbia')),
 ((101, (3, 'Paul', 101)), (102, 'Cambridge')),
 ((101, (3, 'Paul', 101)), (103, 'Naperville')),
 ((102, (4, 'Jimmy', 102)), (100, 'Boca Raton')),
 ((None, (5, 'Janice', None)), (100, 'Boca Raton')),
 ((102, (4, 'Jimmy', 102)), (101, 'Columbia')),
 ((None, (5, 'Janice', None)), (101, 'Columbia')),
 ((102, (4, 'Jimmy', 102)), (102, 'Cambridge')),
 ((None, (5, 'Janice', None)), (102, 'Cambridge')),
 ((102, (4, 'Jimmy', 102)), (103, 'Naperville')),
 ((None, (5, 'Janice', None)), (103, 'Naperville'))]